# RAG Evaluation : Eval Dataset Generation

## Build a synthetic dataset for evaluation

If we do not have access to a real Q&A datset, we can use LLMs to generate synthetic Question and Answer couples based on a given context. This method is a very common approach, the well-known RAGAS package is based on synthetic Q&A. Furthermore, the zero-shot or few-short learning LLM abilities allow to filter out these generated proposals by asking (with a specific prompt) to evaluate (give a grade between 1 and 5) human understandable criteria (like groundedness, relevence,...) in a specific format like JSON. 

In [4]:
import pandas as pd 
data = pd.read_csv("/home/onyxia/work/llm-open-data-insee/data_complete.csv", low_memory=False) #we assume the textual information have already been extracted. 

In [5]:
from db_building import extract_paragraphs
results = extract_paragraphs(data) #gather textual information from the same page together 

0it [00:00, ?it/s]

43226it [01:47, 401.25it/s] 


In [6]:
ds = pd.DataFrame.from_dict(results)
ds.to_csv("insee_documents.csv", index=False)

In [7]:
from tqdm import tqdm
from langchain.docstore.document import Document as LangchainDocument
import pandas as pd

ds = pd.read_csv("insee_documents.csv")
langchain_docs = [LangchainDocument(page_content=doc["document"], metadata={"source": doc["url_source"], "title": doc["title"], "insee_id": doc["id_origin"]}) for _, doc in tqdm(ds.iterrows())]

34567it [00:02, 12286.48it/s]


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, #define the size of the chunck
    chunk_overlap=200, #define the overlapping between following chunks
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in tqdm(langchain_docs):
    docs_processed += text_splitter.split_documents([doc])

  0%|          | 0/34567 [00:00<?, ?it/s]

100%|██████████| 34567/34567 [00:04<00:00, 6925.16it/s] 


In [9]:
len(docs_processed)

90730

Note : Phi3 microsoft SLM is not yet in HF official package version. Loading a Development version is required. 

In [ ]:
#!pip uninstall -y transformers && pip install git+https://github.com/huggingface/transformers
from transformers import BitsAndBytesConfig,  AutoModelForCausalLM, AutoTokenizer,  AutoConfig
import torch

#model_name = "microsoft/Phi-3-mini-128k-instruct" #(to download)
#path_model = "microsoft-Phi-3-mini-128k-instruct" #(to load)
#model.save_pretrained("microsoft-Phi-3-mini-128k-instruct")

model_name = "microsoft/Phi-3-mini-4k-instruct" #smaller model 

#load LLM config 
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
#config.max_position_embeddings = 8096
#load quantization config 
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="float16",
        bnb_4bit_use_double_quant=False,
    )

tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = AutoModelForCausalLM.from_pretrained(
            model_name,
            config=config,
            quantization_config=quantization_config,
            device_map="cuda", 
            torch_dtype="auto", 
            trust_remote_code=True,
        )
model

The aim is to generate **questions** and **answers** based on large **context** (part of INSEE documents). 

In [8]:
QA_generation_prompt = """
<|user|>
Ta tâche consiste à écrire une question factuelle et sa réponse en te basant sur un contexte donné.
Ta question factuelle doit pouvoir être répondue par une information factuelle spécifique et concise tirée du contexte.
Ta question factuelle doit être formulée dans le même style que les questions que les utilisateurs pourraient poser dans un moteur de recherche.
Cela signifie que ta question factuelle NE DOIT PAS mentionner des phrases comme "selon le passage" ou "le contexte".
<|user|>
<|assistant|>
Voici maintenant le contexte.

Contexte : {context}

Question factuelle : (ta question factuelle)
Réponse : (ta réponse à la question factuelle)
<|assistant|>
"""

In [14]:
import numpy as np 
cont_sam = docs_processed[int(np.random.choice(range(len(docs_processed)), 1)[0])]
print(QA_generation_prompt.format(context=cont_sam.page_content))


<|user|>
Ta tâche consiste à écrire une question factuelle et sa réponse en te basant sur un contexte donné.
Ta question factuelle doit pouvoir être répondue par une information factuelle spécifique et concise tirée du contexte.
Ta question factuelle doit être formulée dans le même style que les questions que les utilisateurs pourraient poser dans un moteur de recherche.
Cela signifie que ta question factuelle NE DOIT PAS mentionner des phrases comme "selon le passage" ou "le contexte".
<|user|>
<|assistant|>
Voici maintenant le contexte.

Contexte : En avril 2021, la confiance des ménages dans la situation économique est stable: l’indicateur qui la synthétise se maintient à 94, en dessous de sa moyenne de longue période (100). En avril 2021, la confiance des ménages dans la situation économique est stable: l’indicateur qui la synthétise se maintient à 94, en dessous de sa moyenne de longue période (100). En avril, le solde d’opinion des ménages relatif à leur situation financière fut

In [ ]:
from transformers import pipeline
import torch 
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipeline("text-generation", model = model, tokenizer=tokenizer)

In [10]:
pipe.device

device(type='cuda')

In [11]:
def clean_output(s):
    """ remove unwanted characters """
    return s.replace("\n", "")

In [12]:
import time 
from transformers import pipeline
import random 
generation_args = {
    "max_new_tokens": 300,
    "return_full_text": False,
    "do_sample": False,
}

import torch 
def generate_test_dataset(pipeline, docs_processed, N_GENERATIONS, BATCH_SIZE):
    
    torch.cuda.empty_cache()
    print(f"Generating {N_GENERATIONS} QA couples...")
    t0 = time.time()
    outputs = []
    docs_sampled = random.sample(docs_processed, N_GENERATIONS)
    # Process documents in batches
    batch_contexts = [doc.page_content for doc in docs_sampled]
    # Generate QA couples for the batch
    generation_args["batch_size"] = BATCH_SIZE
    batch_prompts = [QA_generation_prompt.format(context=ctx) for ctx in batch_contexts]
    
    with torch.no_grad():
        generated_out = pipeline(batch_prompts, **generation_args)

    for output, sampled_context in zip(generated_out, docs_sampled):
        output_QA_couple = output[0]["generated_text"]
        question = output_QA_couple.split("Question factuelle : ")[-1].split("Réponse : ")[0]
        answer = output_QA_couple.split("Réponse : ")[-1]

        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": clean_output(question),
                "answer": clean_output(answer),
                "source_doc": sampled_context.metadata["source"],
            }
        )
    print("run in ", time.time() - t0, "s")

    return outputs


In [13]:
outputs = generate_test_dataset(pipe , docs_processed, N_GENERATIONS=200, BATCH_SIZE=32)

Generating 200 QA couples...


You are not running the flash-attention implementation, expect numerical differences.


run in  163.5787217617035 s


In [14]:
display(pd.DataFrame(outputs).head(5))

,context,question,answer,source_doc
0,"Au quatrième trimestre 2021, le PIB français s...",Quelle est l'évolution prévue du PIB français ...,L'évolution prévue du PIB français au premier ...,https://www.insee.fr/fr/information/6215215
1,". Pendant les périodes d’activité partielle, l...",Quels secteurs sont particulièrement exposés a...,Les secteurs particulièrement exposés aux cons...,https://www.insee.fr/fr/statistiques/6216009
2,Près de 200 000 habitants résident sur le Terr...,Quelle était la population du Territoire de la...,La population du Territoire de la Côte Ouest é...,https://www.insee.fr/fr/statistiques/1293858
3,. Un métier pour lequel la part des femmes se ...,Quelles mesures ont été mises en place pour pr...,"Des mesures telles que la loi Génisson (2001),...",https://www.insee.fr/fr/statistiques/1285800
4,Les données de population au 1er janvier 2021 ...,Quand Mayotte a-t-elle commencé à utiliser la ...,Mayotte a commencé à utiliser la méthode commu...,https://www.insee.fr/fr/statistiques/7728787


In [16]:
#SSP Cloud access : mc cp s3/projet-llm-insee-open-data/data/eval_data/q_and_a_insee_200.csv /home/onyxia/work/llm-open-data-insee/data/test
q_and_a_data = pd.DataFrame(outputs)
q_and_a_data.to_csv("q_and_a_insee_200.csv")

### Setup critique agents 
Need to evaluate if the generated questions are relevant
- Groundedness 
- Relevance
- Stand-alone question : Can the question be answerable without the linked document. (Is it a general knowledge question)

These 3 tests are done using prompting and asking the model to share its explanations and then its score. 
The scale is between 1 and 5.
At the end, the remaining questions received 4+

In [24]:
q_and_a_data.head()

,context,question,answer,source_doc
0,"Au quatrième trimestre 2021, le PIB français s...",Quelle est l'évolution prévue du PIB français ...,L'évolution prévue du PIB français au premier ...,https://www.insee.fr/fr/information/6215215
1,". Pendant les périodes d’activité partielle, l...",Quels secteurs sont particulièrement exposés a...,Les secteurs particulièrement exposés aux cons...,https://www.insee.fr/fr/statistiques/6216009
2,Près de 200 000 habitants résident sur le Terr...,Quelle était la population du Territoire de la...,La population du Territoire de la Côte Ouest é...,https://www.insee.fr/fr/statistiques/1293858
3,. Un métier pour lequel la part des femmes se ...,Quelles mesures ont été mises en place pour pr...,"Des mesures telles que la loi Génisson (2001),...",https://www.insee.fr/fr/statistiques/1285800
4,Les données de population au 1er janvier 2021 ...,Quand Mayotte a-t-elle commencé à utiliser la ...,Mayotte a commencé à utiliser la méthode commu...,https://www.insee.fr/fr/statistiques/7728787


In [36]:
question_groundedness_critique_prompt = """
<|user|>
Vous allez recevoir un contexte et une question.
Votre tâche est de fournir une évaluation globale notant dans quelle mesure on peut répondre de manière non ambiguë à la question donnée avec le contexte donné.
Donnez votre réponse sur une échelle de 1 à 5, où 1 signifie que la question n'est pas du tout répondable compte tenu du contexte, et 5 signifie que la question est clairement et sans ambiguïté répondable avec le contexte.

Fournissez votre réponse comme suit :

Évaluation : (votre justification de la note, sous forme de texte)
Note totale : (votre note, entre 1 et 5)

Vous DEVEZ fournir des valeurs pour 'Évaluation :' et 'Note totale :' dans votre réponse et rien d'autre. 

Maintenant, voici la question et le contexte.
Question : {question}
Contexte : {context}
<|end|>
<|assistant|>
"""
question_relevance_critique_prompt  = """
<|user|>
Vous allez recevoir une question.
Votre tâche est de fournir une 'notation globale' représentant à quel point cette question peut être utile pour les agents de l'institut de statistique public français.
Donnez votre réponse sur une échelle de 1 à 5, où 1 signifie que la question n'est pas du tout utile, et 5 signifie que la question est extrêmement utile.

Fournissez votre réponse comme suit :

Évaluation : (votre justification de la note, sous forme de texte)
Note totale : (votre note, sous forme d'un nombre entre 1 et 5)

Vous DEVEZ fournir des valeurs pour 'Évaluation :' et 'Note totale :' dans votre réponse.

Maintenant, voici la question.

Question : {question}
<|end|>
<|assistant|>
"""

question_standalone_critique_prompt = """ 
<|user|>
Vous allez recevoir une question.
Votre tâche est de fournir une 'notation globale' représentant à quel point cette question est indépendante du contexte.
Donnez votre réponse sur une échelle de 1 à 5, où 1 signifie que la question dépend d'informations supplémentaires pour être comprise, et 5 signifie que la question a du sens par elle-même.
Par exemple, si la question fait référence à un cadre particulier, comme 'dans le contexte' ou 'dans le document', la note doit être de 1.
Les questions peuvent contenir des noms techniques obscurs ou des acronymes comme INSEE ou du vocabulaire propre au statistiques et obtenir tout de même une note de 5 : il suffit simplement qu'un agent d'un institut de statistique public ayant accès à la documentation comprenne de quoi parle la question.

Fournissez votre réponse comme suit :

Évaluation : (votre justification de la note, sous forme de texte)
Note totale : (votre note, sous forme d'un nombre entre 1 et 5)

Vous DEVEZ fournir des valeurs pour 'Évaluation :' et 'Note totale :' dans votre réponse.

Maintenant, voici la question.

Question : {question}\n
<|end|>
<|assistant|>
"""


In [78]:
import re
import pandas as pd
from tqdm import tqdm

def extract_btw_tag(text, tag_1, tag_2):
    pattern = f'{tag_1}(.*?){tag_2}'
    return re.findall(pattern, text)[0] if re.findall(pattern, text) else None
    
def extract_score_eval(text):
    eval_text = extract_btw_tag(text, tag_1="Évaluation : ", tag_2="\n")
    score_text = extract_btw_tag(text, tag_1="Note totale : ", tag_2="\n")
    score = int(score_text) if score_text else None
    return score, eval_text 

def critique_Q_and_A(pipeline, dataset, args, k=2):

    data = dataset.copy()

    num_rows = len(data)

    for batch_start in tqdm(range(0, num_rows, k)):
        batch_end = min(batch_start + k, num_rows)  # Adjust batch end to handle the last incomplete batch
        batch_prompts = []

        for idx in range(batch_start, batch_end):
            row = data.iloc[idx]
            batch_prompts.extend([
                question_groundedness_critique_prompt.format(context=row.context, question=row.question), 
                question_relevance_critique_prompt.format(question=row.question), 
                question_standalone_critique_prompt.format(question=row.question)
            ])

        outputs = pipeline(batch_prompts, **args)

        try:
            for j, idx in enumerate(range(batch_start, batch_end)):
                metrics = {}
                metrics["groundedness_score"], metrics["groundedness_eval"] = extract_score_eval(outputs[3*j][0]['generated_text']) 
                metrics["relevance_score"], metrics["relevance_eval"] = extract_score_eval(outputs[3*j + 1][0]['generated_text'])
                metrics["standalone_score"], metrics["standalone_eval"] = extract_score_eval(outputs[3*j + 2][0]['generated_text'])
                
                data.loc[idx, list(metrics.keys())] = list(metrics.values())

        except Exception as e:
            print('Error:', e)
            continue
    
    return data 


In [28]:
row = q_and_a_data.iloc[0]
prompt_final = question_groundedness_critique_prompt.format(context=row.context, question=row.question)
prompt_final

"\n<|user|>\nVous allez recevoir un contexte et une question.\nVotre tâche est de fournir une évaluation globale notant dans quelle mesure on peut répondre de manière non ambiguë à la question donnée avec le contexte donné.\nDonnez votre réponse sur une échelle de 1 à 5, où 1 signifie que la question n'est pas du tout répondable compte tenu du contexte, et 5 signifie que la question est clairement et sans ambiguïté répondable avec le contexte.\n\nFournissez votre réponse comme suit :\n\nÉvaluation : (votre justification de la note, sous forme de texte)\nNote totale : (votre note, entre 1 et 5)\n\nVous DEVEZ fournir des valeurs pour 'Évaluation :' et 'Note totale :' dans votre réponse et rien d'autre. \n\nMaintenant, voici la question et le contexte.\nQuestion : Quelle est l'évolution prévue du PIB français au premier trimestre 2022 par rapport au quatrième trimestre 2021?\nContexte : Au quatrième trimestre 2021, le PIB français s’est situé à 0,9% au-dessus de son niveau d’avant la cris

In [81]:
generated_questions = critique_Q_and_A(pipe, q_and_a_data, generation_args, k=10)

 85%|████████▌ | 17/20 [07:36<01:21, 27.17s/it]

Error: invalid literal for int() with base 10: '3/5'


 90%|█████████ | 18/20 [08:03<00:54, 27.07s/it]

Error: invalid literal for int() with base 10: '4/5'


100%|██████████| 20/20 [08:56<00:00, 26.84s/it]


In [83]:
generated_questions.head(1)

,context,question,answer,source_doc,groundedness_score,groundedness_eval,relevance_score,relevance_eval,standalone_score,standalone_eval
0,"Au quatrième trimestre 2021, le PIB français s’est situé à 0,9% au-dessus de son niveau d’avant la crise sanitaire, poursuivant son rattrapage dans plusieurs secteurs des services (commerce, transports, services aux ménages...). Les trois premiers mois de l’année 2022 apparaissent plus contrastés. Lemois de janvier a été caractérisé par la dégradation de la situation sanitaire (vague Omicron) et ses conséquences (recours accru au télétravail, mise en place du passe vaccinal, jauges pour certains évènements, etc.). Enfévrier, la plupart des restrictions ont été progressivement levées, et les enquêtes de conjoncture laissaient alors augurer une réaccélération de l’activité. Lemois de mars est quant à lui marqué par les premières conséquences du déclenchement de la guerre en Ukraine, s’ajoutant aux difficultés d’offre (difficultés d’approvisionnement, difficultés de recrutement) et aux tensions sur les prix qui préexistaient. Dans ce contexte, l’évolution du PIB français au premier trimestre 2022 serait le reflet, en moyenne, de ces évolutions contrastées. Ilralentirait ainsi, avec une progression prévue de +0,3% (après +0,7% au quatrième trimestre). Après une progression annuelle du PIB de +7,0% en 2021 par rapport à 2020, l’acquis de croissance pour l’année 2022 à l’issue du premier trimestre –c’est-à-dire la croissance annuelle qui serait observée si le PIB trimestriel restait stable sur le reste de l’année– serait alors de +2,7%...",Quelle est l'évolution prévue du PIB français au premier trimestre 2022 par rapport au quatrième trimestre 2021?,"L'évolution prévue du PIB français au premier trimestre 2022 serait de +0,3%, après une augmentation de +0,7% au quatrième trimestre 2021.",https://www.insee.fr/fr/information/6215215,None,Le contexte fournit des informations sur les conditions économiques et les facteurs qui ont influencé le PIB français au quatrième trimestre 2021 et au début de l'année 2022. Il mentionne également une estimation de l'évolution du PIB pour le premier trimestre 2022.,5.0,"Cette question est très pertinente pour les agents de l'institut de statistique public français car elle concerne une analyse économique directe et spécifique qui pourrait être utilisée pour des prévisions, des rapports ou des analyses économiques. Elle demande une connaissance des tendances économiques et des données historiques, ce qui est au cœur de leur travail.",5.0,"La question mentionne des périodes spécifiques (premier trimestre 2022 et quatrième trimestre 2021) et un indicateur économique (PIB français), qui sont des informations qui peuvent être trouvées dans des données économiques publiques ou des rapports d'institutions comme l'INSEE. Il n'est pas nécessaire d'avoir un contexte plus large pour comprendre la question."


Filter out bad questions based on our critique agent scores

In [84]:
import pandas as pd

pd.set_option("display.max_colwidth", None)

print("Evaluation dataset before filtering:")

display(
    generated_questions[
        [
            "question", # question generated based on given context
            "answer", # answer generated based on given context (and generated question)
            "groundedness_score", #score 
            "relevance_score", #score 
            "standalone_score", #score 
        ]
    ]
)

#Select only the most relevant questions
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    & (generated_questions["relevance_score"] >= 4)
    & (generated_questions["standalone_score"] >= 4)
]
print("number of selected Q&A couple : ", len(generated_questions))

print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)


Evaluation dataset before filtering:


,question,answer,groundedness_score,relevance_score,standalone_score
0,Quelle est l'évolution prévue du PIB français au premier trimestre 2022 par rapport au quatrième trimestre 2021?,"L'évolution prévue du PIB français au premier trimestre 2022 serait de +0,3%, après une augmentation de +0,7% au quatrième trimestre 2021.",None,5.0,5.0
1,Quels secteurs sont particulièrement exposés aux conséquences de la crise Covid-19 dans l'est de la région et au sud de Clermont-Ferrand?,"Les secteurs particulièrement exposés aux conséquences de la crise Covid-19 dans l'est de la région et au sud de Clermont-Ferrand sont ceux de montagne, plasturgie et décolletage.",3,4.0,4.0
2,Quelle était la population du Territoire de la Côte Ouest au 1er janvier 2006 et quel était le taux de croissance annuel de la population depuis 1999?,"La population du Territoire de la Côte Ouest était de 200 000 habitants au 1er janvier 2006. La population avait connu une croissance annuelle de 1,5 % depuis 1999.",4,5.0,5.0
3,Quelles mesures ont été mises en place pour promouvoir la mixité professionnelle et l'égalité des sexes dans le marché du travail en France?,"Des mesures telles que la loi Génisson (2001), l'accord national interprofessionnel sur la mixité et l'égalité professionnelle (2004), la création du contrat pour la mixité des emplois et l'égalité (2011), l'instauration progressive de quotas de femmes dans les conseils d'administration et de surveillance des entreprises (2011), et la loi cadre pour l’égalité réelle entre les femmes et les hommes (2014) ont été mises en place pour promouvoir la mixité professionnelle et l'égalité des sexes dans le marché du travail en France.",5,4.0,4.0
4,"Quand Mayotte a-t-elle commencé à utiliser la méthode commune de recensement de la population, et quand devrait-on attendre la première population légale authentifiée après cette méthode?",Mayotte a commencé à utiliser la méthode commune de recensement de la population en 2021. La première population légale authentifiée après l'adoption de cette méthode devrait être attendue à la fin de 2025.,5,4.0,4.0
...,...,...,...,...,...
195,Quelle est la baisse la plus importante du chômage dans les régions métropolitaines au quatrième trimestre 2016 et dans quelle région cela s'est produit-elle?,"La baisse la plus importante du chômage dans les régions métropolitaines au quatrième trimestre 2016 s'est produite dans la Meuse, Meurthe-et-Moselle et les Vosges avec une baisse de 2,1%, 1,2% et 0,9% respectivement.",5,5.0,2.0
196,Quelle est la différence entre le taux de natalité du Grand Est et celui de la France métropolitaine en 2015?,"En 2015, le taux de natalité du Grand Est était de 10,8 naissances pour 1000 habitants, soit un taux de natalité sensiblement inférieur à celui de la France métropolitaine qui était de 11,8‰.",3,4.0,5.0
197,"Quelle industrie a connu une augmentation de +2,7% en décembre, après une baisse de -1,1%?","L'industrie pharmaceutique a connu une augmentation de +2,7% en décembre, après une baisse de -1,1%.",5,4.0,5.0
198,Quelle est la différence de pourcentage entre la baisse des entreprises immatriculées en Midi-Pyrénées et la baisse du nombre total d'entreprises créées en France au 2e trimestre 2013?,La différence de pourcentage est de 12% (8% pour les auto-entreprises en Midi-Pyrénées et 3% pour le nombre total d'entreprises en France).,2,4.0,5.0


number of selected Q&A couple :  77
Final evaluation dataset:


,question,answer,groundedness_score,relevance_score,standalone_score
2,Quelle était la population du Territoire de la Côte Ouest au 1er janvier 2006 et quel était le taux de croissance annuel de la population depuis 1999?,"La population du Territoire de la Côte Ouest était de 200 000 habitants au 1er janvier 2006. La population avait connu une croissance annuelle de 1,5 % depuis 1999.",4,5.0,5.0
3,Quelles mesures ont été mises en place pour promouvoir la mixité professionnelle et l'égalité des sexes dans le marché du travail en France?,"Des mesures telles que la loi Génisson (2001), l'accord national interprofessionnel sur la mixité et l'égalité professionnelle (2004), la création du contrat pour la mixité des emplois et l'égalité (2011), l'instauration progressive de quotas de femmes dans les conseils d'administration et de surveillance des entreprises (2011), et la loi cadre pour l’égalité réelle entre les femmes et les hommes (2014) ont été mises en place pour promouvoir la mixité professionnelle et l'égalité des sexes dans le marché du travail en France.",5,4.0,4.0
4,"Quand Mayotte a-t-elle commencé à utiliser la méthode commune de recensement de la population, et quand devrait-on attendre la première population légale authentifiée après cette méthode?",Mayotte a commencé à utiliser la méthode commune de recensement de la population en 2021. La première population légale authentifiée après l'adoption de cette méthode devrait être attendue à la fin de 2025.,5,4.0,4.0
10,Combien d'emplois sont considérés comme liés au tourisme dans les secteurs d'activités partiellement touristiques?,"Pour les secteurs d'activités partiellement touristiques, l'emploi lié au tourisme est estimé en soustrayant à l'emploi total un emploi théorique lié aux résidents.",4,4.0,4.0
13,Quelle est la différence de nombre de décès entre la période du 1er mars au 30 avril 2020 et le même laps de temps en 2019 dans la région de Bourgogne-Franche-Comté?,"La différence de nombre de décès entre le 1er mars et le 30 avril 2020 et le même laps de temps en 2019 dans la région de Bourgogne-Franche-Comté est de 4450 décès, soit une augmentation de 26%.",5,5.0,5.0
...,...,...,...,...,...
176,Quelle est la différence de rémunération moyenne entre les accords décentralisés et les accords de branche en France?,"La différence de rémunération moyenne entre les accords décentralisés et les accords de branche en France est d'environ 9% (5,5% pour les accords de branche et 13,5% pour les accords décentralisés).",5,4.0,4.0
180,"Quelle est la progression de l'emploi salarié dans le secteur tertiaire marchand hors intérim, en particulier dans les activités spécialisées, scientifiques et techniques, depuis le dernier trimestre?","La progression de l'emploi salarié dans le secteur tertiaire marchand hors intérim, en particulier dans les activités spécialisées, scientifiques et techniques, est de +0,9% d'un trimestre à l'autre.",5,4.0,5.0
188,Quelle est la tendance démographique de la population régionale de Bourgogne-Franche-Comté depuis 2015 et quelles sont les principales causes de cette tendance?,"La population régionale de Bourgogne-Franche-Comté a une tendance à la baisse depuis 2015. Cette tendance est principalement due à une diminution marquée du nombre de naissances et à une augmentation sans précédent du nombre de décès. Le solde naturel n'a jamais été aussi négatif dans cette région, ce qui contribue à l'accroissement naturel négatif de -2,9 pour mille habitants.",5,5.0,4.0
190,"Quels sont les sujets principaux traités dans le rapport annuel ""L'Économie française - Comptes et dossiers""?","Les sujets principaux traités dans le rapport annuel ""L'Économie française - Comptes et dossiers"" sont les mouvements essentiels ayant affecté les économies française et mondiale au cours de l'année écoulée.",4,5.0,5.0


In [87]:
generated_questions.to_csv("eval_dataset.csv", index=False)

In [86]:
!mc cp /home/onyxia/work/eval_dataset.csv s3/projet-llm-insee-open-data/data/eval_data/

...ataset.csv: 240.80 KiB / 240.80 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 1.51 MiB/s 0s